In [1]:
### Machine Running library install
from sklearn.neighbors import KNeighborsClassifier


from sklearn.linear_model import LogisticRegression

### 선형, 다중, 다항 회귀모델 라이브러리 정의
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

### 앙상블 모델
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor

### visualization library
import matplotlib.pyplot as plt

# Definition of the NumPy library
import numpy as np

### Definition of Library (Preprocessing Library)
from sklearn.model_selection import train_test_split

### 평가 라이브러리 정의
# 평균절대오차(MAE)
from sklearn.metrics import mean_absolute_error
# 평균제곱오차(MSE)
from sklearn.metrics import mean_squared_error
# 결정계수(R2-score)
from sklearn.metrics import r2_score

# - 변환기 모델(클래스) 라이브러리 정의하기
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

import pandas as pd 

import seaborn as sns


### 라이브러리 정의
from scipy.stats import spearmanr

### 라이브러리 정의
from scipy.stats import pearsonr

### 하이퍼파라미터 튜닝 모델(클래스) 정의하기
from sklearn.model_selection import GridSearchCV

plt.rc("font", family="Malgun Gothic")

plt.rcParams["axes.unicode_minus"] = False

### 경고 메시지 없애기
# - 사이킷런 버전에 따라 오류가 아니니 안내(경고)메시지가 자주 나타남
# - 안내(경고) 메시지 없이 실행할 수 있도록 처리
from sklearn import set_config
set_config(display="text")


In [ ]:
"""
<분류분석>
 - 종속변수 범주의 갯수에 따라 이진분류와 다중분류로 구분됨
 - 이진분류: 종속변수가 2개인 경우(둘 중 하나 예측하기)
 - 다중분류: 종속변수가 3개 이상인 경우(여러개 중 하나 예측하기)
           : (일부 다중분류 라이브러리에서는 종속변수가 2개 이상인 경우도 포함됨)
 - 분류분석 모델: KNN, 로지스틱레그레이션, 결정트리, 앙상블모델들
                : 주로 사용되는 모델 -> 앙상블 모델
"""

In [2]:
### 생선분류 데이터셋.csv 데이터 불러들이기
# - 변수명: fish 사용

file_path = "./data/05_생선_분류_데이터셋.csv"
fish = pd.read_csv(file_path)
print(fish.info())
print(fish.describe())
print(fish.head(2))

### 무게 데이터의 최소값(min)에 이상치로 보이는 0값이 있음
# - 이상치 처리를 선행해야 함.
# - 단, 분류모델을 한번 리뷰하기 위해 결측, 이상, 중복 처리가 되었다는 가정하에 진행

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Species   159 non-null    object 
 1   Weight    159 non-null    float64
 2   Length    159 non-null    float64
 3   Diagonal  159 non-null    float64
 4   Height    159 non-null    float64
 5   Width     159 non-null    float64
dtypes: float64(5), object(1)
memory usage: 7.6+ KB
None
            Weight      Length    Diagonal      Height       Width
count   159.000000  159.000000  159.000000  159.000000  159.000000
mean    398.326415   28.415723   31.227044    8.970994    4.417486
std     357.978317   10.716328   11.610246    4.286208    1.685804
min       0.000000    8.400000    8.800000    1.728400    1.047600
25%     120.000000   21.000000   23.150000    5.944800    3.385650
50%     273.000000   27.300000   29.400000    7.786000    4.248500
75%     650.000000   35.500000   39.650000   12.365900    

In [ ]:
"""
* 특성 설명
 - Species: 생선 종류
 - Weight: 생선 무게 (g)
 - Length: 생선 길이 (cm)
 - Diagonal: 생선 대각선 길이 (cm)
 - Height: 생선 높이 (cm)
 - Width: 생선 두께 (cm)
 
* 분석 주제
 - 생선의 무게, 길이, 대각선길이, 높이, 두께 데이터를 이용하여
 - 생선의 종류 분류하기
 
* 분석을 위한 데이터
 - 독립변수: 무게, 길이, 대각선길이, 높이, 두께
 - 종속변수: 생선 종류
"""

### 독립변수와 종속변수로 분리하기

In [3]:
### 변립변수명: fish_input, 종속변수명: fish_target
# - numpy의 array 타입으로 처리해 주세요.

fish_input = fish.iloc[:, 1:].to_numpy()

fish_target = fish["Species"].to_numpy()

print(fish_input.shape)
print(fish_target.shape)


(159, 5)
(159,)


### 훈련 : 테스트 = 7.5 : 2.5로 분리하기

In [4]:
### 변수명: train_input, train_target, test_input, test_target

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, test_size=0.25, random_state=42, 
    ### 종속변수의 범주 비율을 편향을 최소화하여 비율대비 섞도록 처리
    stratify=fish_target
)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)


(119, 5) (119,)
(40, 5) (40,)


### Standard 스케일링 처리하기

In [5]:
### 독립변수명: train_scaled, test_scaled

scaler = StandardScaler()

scaler.fit(train_input)

train_scaled = scaler.transform(train_input)
test_scaled = scaler.transform(test_input)

print(train_scaled.shape, train_target.shape)
print(test_scaled.shape, test_target.shape)

(119, 5) (119,)
(40, 5) (40,)


### KNN(최근접이웃) 분류모델로 훈련하기

In [6]:
### KNN 분류모델을 이용하여 분류하기
# - 모델 변수명: k
# - 이웃의 갯수: 3개 사용
# - 과적합 여부 확인

kn = KNeighborsClassifier(n_neighbors=9)

kn.fit(train_scaled, train_target)

train_score = kn.score(train_scaled, train_target)
test_score = kn.score(test_scaled, test_target)

print(f"훈련 정확도{train_score}, 테스트 정확도: {test_score}, 과적합 여부: {train_score - test_score}")


훈련 정확도0.773109243697479, 테스트 정확도: 0.7, 과적합 여부: 0.07310924369747906


In [7]:
### KNN 모델이 사용한 종속변수 확인하기
# - 모델이 사용하는 종속변수 범주의 순서
#   ['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish']
kn.classes_

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [8]:
### 테스트 데이터 상위 5개 데이터로 예측하기
test_pred = kn.predict(test_scaled[:5])
test_pred

array(['Perch', 'Perch', 'Perch', 'Roach', 'Parkki'], dtype=object)

In [9]:
### 실제 정답 확인하기
test_target[:5]

# - 예측: ['Perch', 'Perch', 'Perch', 'Roach', 'Parkki']
# - 정답: ['Roach', 'Perch', 'Perch', 'Parkki', 'Parkki']

array(['Roach', 'Perch', 'Perch', 'Parkki', 'Parkki'], dtype=object)

In [10]:
### KNN 모델이 분류한 확률 확인하기
kn.predict_proba(test_scaled[:5])

# - 모델이 사용하는 종속변수 범주의 순서
#   ['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish']
#       0         1        2        3       4        5          6 
 
# - 예측: ['Perch', 'Perch', 'Perch', 'Roach', 'Parkki']

array([[0.        , 0.        , 0.66666667, 0.        , 0.33333333,
        0.        , 0.        ],
       [0.        , 0.        , 0.77777778, 0.        , 0.22222222,
        0.        , 0.        ],
       [0.        , 0.        , 0.55555556, 0.        , 0.44444444,
        0.        , 0.        ],
       [0.        , 0.33333333, 0.22222222, 0.        , 0.44444444,
        0.        , 0.        ],
       [0.        , 0.44444444, 0.22222222, 0.        , 0.33333333,
        0.        , 0.        ]])

### 로지스틱 리그레이션(LogisticRegression) 모델

In [ ]:
"""
<로지스틱 리그레이션 모델>
 - 이름은 회귀이지만, 분류분석에서 사용되는 모델임
 - 내부적으로 사용되는 산술식은 "다항회귀" 모델의 알고리즘을 사용함
 - 분류분석에서 -> 이진분류 및 다중분류에서 모두 사용됨
 - 분류모델 중 초기에 사용되었던 모델임
"""

### 이진분류 해보기

In [ ]:
"""
<이진분류>
 - 종속변수의 범주의 갯수는 2개만 허용됨

<이진분류 데이터 구성>
 - 생선의 종류 Bream 및 Smelt 값에 대해서만 분류해보기
 - 데이터 재가공이 필요함
"""

In [ ]:
### 현재 기준 독립변수 및 종속변수 원보: *_scaled, *_target

### 훈련 독립변수명: train_scaled_bream_smelt, train_scaled_bream_smelt_target
### 테스트 독립변수명: test_scaled_bream_smelt, test_scaled_bream_smelt_target

### 종속변수를 이용하여 Bream 및 Smelt의 index 위치 확인
train_index = (train_target == "Bream") | (train_target == "Smelt")
test_index  = (test_target == "Bream")  | (test_target == "Smelt")

### 훈련 독립 및 종속변수 추출하기
train_scaled_bream_smelt        = train_scaled[train_index]
train_scaled_bream_smelt_target = train_target[train_index]

### 테스트 독립 및 종속변수 추출하기
test_scaled_bream_smelt        = test_scaled[test_index]
test_scaled_bream_smelt_target = test_target[test_index]

print(train_scaled_bream_smelt.shape, train_scaled_bream_smelt_target.shape)
print(test_scaled_bream_smelt.shape, test_scaled_bream_smelt_target.shape)

(36, 5) (36,)
(13, 5) (13,)


### 로지스틱 리그레이션 모델로 훈련하기

In [ ]:
"""
 - 사용 패키지: sklearn.linear_model
 - 사용 모델  : LogisticRegression
 
 - 모델 변수명: lr
 - 과적합 여부 확인
 - 예측: 상위 5개 데이터로 예측
 - 정답과 비교해보기
"""

In [12]:
lr = LogisticRegression()

lr.fit(train_scaled_bream_smelt, train_scaled_bream_smelt_target)

train_score = lr.score(train_scaled_bream_smelt, train_scaled_bream_smelt_target)
test_score  = lr.score(test_scaled_bream_smelt, test_scaled_bream_smelt_target)

print(f"훈련 정확도{train_score}, 테스트 정확도: {test_score}, 과적합 여부: {train_score - test_score}")

test_pred = lr.predict(test_scaled_bream_smelt[:5])
print(f"예측결과: {test_pred}")
print(f"실제정답: {test_scaled_bream_smelt_target[:5]}")

# - 예측: ['Bream' 'Bream' 'Bream' 'Bream' 'Bream']
# - 정답: ['Bream' 'Bream' 'Bream' 'Bream' 'Bream']

print(lr.predict_proba(test_scaled_bream_smelt[:5]))

훈련 정확도1.0, 테스트 정확도: 1.0, 과적합 여부: 0.0
예측결과: ['Bream' 'Bream' 'Bream' 'Bream' 'Bream']
실제정답: ['Bream' 'Bream' 'Bream' 'Bream' 'Bream']
[[0.97463552 0.02536448]
 [0.9953484  0.0046516 ]
 [0.99562915 0.00437085]
 [0.99870965 0.00129035]
 [0.99476359 0.00523641]]


### 로지스틱리그레이션 모델 하이퍼파라미터 튜닝하기

In [ ]:
"""
<하이퍼파라미터>
 - 튜닝에 사용되는 기본적인 속성: C(규제강도), max_iter(훈련 반복횟수)
 - C(규제강도)
  : 릿지 또는 라쏘에서 사용한 alpha의 규제강도와 동일한 개념
  : 복잡도를 높이거나 낮추어서 과적합 해소
  : 기본값(default): 1
  : 값의 범위: 0 이상~
  : C의 값이 커질수록 강도가 약해짐
  
 - max_iter(훈련 반복횟수)
  : 훈련을 여러번 반복하여 훈련의 정확도를 높임
  : 기본값(default): 100
  : 값의 범위: 1 이상~
  
 ***** 하이퍼파라미터 튜닝 이후 좋아질수도 or 그렇지 않을수도 있습니다.
       !!! 꼭~ 좋아진다는 편견 위험
"""

In [13]:
### 규제강도는 0.5로, 훈련 반복횟수는 100번
# 과적합여부 확인까지 수행해주세요
lr = LogisticRegression(C=0.014, max_iter=100)

lr.fit(train_scaled_bream_smelt, train_scaled_bream_smelt_target)

train_score = lr.score(train_scaled_bream_smelt, train_scaled_bream_smelt_target)
test_score  = lr.score(test_scaled_bream_smelt, test_scaled_bream_smelt_target)

print(f"훈련 정확도{train_score}, 테스트 정확도: {test_score}, 과적합 여부: {train_score - test_score}")

test_pred = lr.predict(test_scaled_bream_smelt[:5])
print(f"예측결과: {test_pred}")
print(f"실제정답: {test_scaled_bream_smelt_target[:5]}")

# - 예측: ['Bream' 'Bream' 'Bream' 'Bream' 'Bream']
# - 정답: ['Bream' 'Bream' 'Bream' 'Bream' 'Bream']

print(lr.predict_proba(test_scaled_bream_smelt[:5]))

###(해석)
# - 이진분류 수행시 규제강도는 0.014, 훈련 반복횟수는 100회 일 때
# - 일반화가 되었으며, 훈련 정확도 0.97 매우 높은 성능을 발휘하고 있음

훈련 정확도0.9722222222222222, 테스트 정확도: 0.9230769230769231, 과적합 여부: 0.04914529914529908
예측결과: ['Bream' 'Bream' 'Bream' 'Bream' 'Bream']
실제정답: ['Bream' 'Bream' 'Bream' 'Bream' 'Bream']
[[0.77774007 0.22225993]
 [0.83290025 0.16709975]
 [0.83488237 0.16511763]
 [0.86743928 0.13256072]
 [0.82962258 0.17037742]]


### 다중 분류 수행하기

In [ ]:
### 종속변수의 범주 7개 모두 포함된 데이터 사용
# - train_scaled, test_scaled, train_target, test_target
# - 1. 하이퍼파라미터 튜닝 없이 과적합 여부 확인
# - 2. 하이퍼파라미터 튜닝 이후 과적합 여부 확인
#    -> 하이퍼파라미터 훈련횟수는 100번, 규제강도는 찾아주세요.

In [14]:
lr = LogisticRegression()

lr.fit(train_scaled, train_target)

train_score = lr.score(train_scaled, train_target)
test_score  = lr.score(test_scaled, test_target)

print(f"훈련 정확도{train_score}, 테스트 정확도: {test_score}, 과적합 여부: {train_score - test_score}")

test_pred = lr.predict(test_scaled[:5])
print(f"예측결과: {test_pred}")
print(f"실제정답: {test_target[:5]}")


훈련 정확도0.8151260504201681, 테스트 정확도: 0.825, 과적합 여부: -0.009873949579831898
예측결과: ['Perch' 'Perch' 'Perch' 'Parkki' 'Parkki']
실제정답: ['Roach' 'Perch' 'Perch' 'Parkki' 'Parkki']


In [16]:
lr = LogisticRegression(C=1000, max_iter=150)

lr.fit(train_scaled, train_target)

train_score = lr.score(train_scaled, train_target)
test_score  = lr.score(test_scaled, test_target)

print(f"훈련 정확도{train_score}, 테스트 정확도: {test_score}, 과적합 여부: {train_score - test_score}")

test_pred = lr.predict(test_scaled[:5])
print(f"예측결과: {test_pred}")
print(f"실제정답: {test_target[:5]}")

훈련 정확도0.9915966386554622, 테스트 정확도: 0.975, 과적합 여부: 0.016596638655462237
예측결과: ['Roach' 'Perch' 'Perch' 'Parkki' 'Parkki']
실제정답: ['Roach' 'Perch' 'Perch' 'Parkki' 'Parkki']
